# Evaluating Regression Models for Student Grade Prediction

**Download the dataset**

In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("stealthtechnologies/predict-student-performance-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\theki\.cache\kagglehub\datasets\stealthtechnologies\predict-student-performance-dataset\versions\1


**reading the dataset into a panda dataframe**

In [2]:
import pandas as pd
import sklearn

student_performance_dataset=pd.read_csv(path+'\\data.csv')
student_performance_dataset.head(20)


,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
0,0.95822,3.4,8.2,53.0,47.0
1,0.85566,3.2,5.9,55.0,35.0
2,0.68025,3.2,9.3,41.0,32.0
3,0.25936,3.2,8.2,47.0,34.0
4,0.60447,3.8,10.0,75.0,33.0
5,0.98320,3.4,9.0,47.0,51.0
6,0.56648,7.9,8.1,63.0,54.0
7,0.93487,1.4,8.0,47.0,34.0
8,0.46660,5.4,8.8,67.0,39.0
9,0.62130,1.4,9.6,42.0,34.0


In [ ]:
#characteristics of the dataset
student_performance_dataset.describe()


,Socioeconomic Score,Study Hours,Sleep Hours,Attendance (%),Grades
count,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000
mean,0.552274,4.560807,8.047262,58.536023,40.691643
std,0.261272,1.897581,1.370700,11.675287,9.467358
min,0.101280,0.800000,4.800000,40.000000,32.000000
25%,0.322118,3.475000,7.000000,49.000000,34.000000
50%,0.545945,3.900000,8.400000,57.000000,35.000000
75%,0.789610,5.900000,9.100000,66.000000,47.000000
max,0.999820,10.000000,10.000000,100.000000,91.000000


**Assigning the features and target**

In [ ]:
#assigning features and target
X=student_performance_dataset.drop(columns=['Grades'])
y=student_performance_dataset['Grades']

**Ridge regression and linear regression**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.metrics import mean_squared_error as mse_func
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.preprocessing import StandardScaler

results = {
    'Linear Regression': [],
}

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# Train Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)

# Evaluate Linear Regression model
results['Linear Regression'].append({
    'MAE': mean_absolute_error(y_test, y_pred_linear),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_linear)),

})

resultsDF=pd.DataFrame(results)
print(resultsDF)


results={
    'Ridge Regression': []
}

#normalizing the features for ridge regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)

#trying different alpha values
for i in np.arange(1,5 , 1):
    # Train Ridge Regression model
    ridge_model = Ridge(alpha=i)  
    ridge_model.fit(X_train_scaled, y_train)
    y_pred_ridge = ridge_model.predict(X_test_scaled)

    # Evaluate Ridge Regression model
    results['Ridge Regression'].append({
    f'MAE_{i}': mean_absolute_error(y_test, y_pred_ridge),
    f'RMSE_{i}': np.sqrt(mean_squared_error(y_test, y_pred_ridge))
})

# Convert results to DataFrames for easier analysis
resultsDF=pd.DataFrame(results)
#resultsDF=resultsDF.T
print(resultsDF)
 

                                   Linear Regression
0  {'MAE': 3.3575324043623493, 'RMSE': 4.27885573...
                                    Ridge Regression
0  {'MAE_1': 3.513323777928262, 'RMSE_1': 4.32410...
1  {'MAE_2': 3.5123039393030666, 'RMSE_2': 4.3230...
2  {'MAE_3': 3.5112866525585793, 'RMSE_3': 4.3219...
3  {'MAE_4': 3.5103156151486976, 'RMSE_4': 4.3208...


In [6]:
#check correlation between features
print(pd.DataFrame(X_train_scaled).corr())

          0         1         2         3
0  1.000000  0.003811  0.005276 -0.017665
1  0.003811  1.000000 -0.155389  0.467076
2  0.005276 -0.155389  1.000000 -0.091771
3 -0.017665  0.467076 -0.091771  1.000000


In [7]:
#check correlation between features and target

print(pd.DataFrame(X_train_scaled).corrwith(pd.Series(y_train)))

0    0.030308
1    0.014544
2    0.012408
3    0.006617
dtype: float64


**Polynomial regression**

In [8]:
# Train Polynomial Regression model
polynomial_features = PolynomialFeatures(degree=3)  # Change degree as needed
X_train_poly = polynomial_features.fit_transform(X_train)
X_test_poly = polynomial_features.transform(X_test)

polynomial_model = LinearRegression()
polynomial_model.fit(X_train_poly, y_train)
y_pred_poly = polynomial_model.predict(X_test_poly)

# Evaluate Polynomial Regression model
resultsPOLY=({
    'MAE': mean_absolute_error(y_test, y_pred_poly),
    'MSE': mean_squared_error(y_test, y_pred_poly),
})
resultsPOLY

{'MAE': 1.3244508681486051, 'MSE': 2.93688173448457}

In [9]:
#the cross validation helps deciding the gerelization of the model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
polynomial_features = PolynomialFeatures(degree=3)  # Change degree as needed
X_poly = polynomial_features.fit_transform(X)

mse_scores = cross_val_score(polynomial_model, X_poly,y, cv=5, scoring='neg_mean_squared_error')
mae_scores = cross_val_score(polynomial_model, X_poly, y, cv=5, scoring=make_scorer(mean_absolute_error))

print("Average MSE:", -np.mean(mse_scores))  
print("Average MAE:", np.mean(mae_scores))

Average MSE: 2.6378203117753705
Average MAE: 1.2561881203799192


**Regression using decision trees**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

#testing the decision tree model
dt_regressor = DecisionTreeRegressor(max_depth=6, random_state=5)  # Adjust max_depth to prevent overfitting
dt_regressor.fit(X_train, y_train)
y_pred_dt = dt_regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_dt)
mse = mean_squared_error(y_test, y_pred_dt)


print(f"MAE: {mae}")
print(f"MSE: {mse}")


MAE: 1.1928455490704972
MSE: 3.2384842297665846


In [83]:
#cross validation performance
mse_scores = cross_val_score(dt_regressor, X,y, cv=5, scoring='neg_mean_squared_error')
mae_scores = cross_val_score(dt_regressor, X, y, cv=5, scoring=make_scorer(mean_absolute_error))

print("Average MAE:", np.mean(mae_scores))
print("Average MSE:", -np.mean(mse_scores))  


Average MAE: 1.2882642897295362
Average MSE: 3.9973016393980365


**Random forests**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#initialize and train
rf_regressor = RandomForestRegressor(n_estimators=200, max_depth=6, random_state=5)
rf_regressor.fit(X_train, y_train)
#Predictions
y_pred = rf_regressor.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)



MAE: 1.0036993792411915
MSE: 1.9342991298902614


In [80]:
#cross validation with Random forests
rf_scores = cross_val_score(rf_regressor, X, y, cv=5, scoring=make_scorer(mean_absolute_error))

# Print the average MSE
print("Average MAE (Random Forest):", np.mean(rf_scores))


Average MAE (Random Forest): 1.0071432658889945
